In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import VisualBertForPreTraining, BertTokenizer, VisualBertModel
import pickle
import pandas as pd


c:\Users\arman\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Define the training dataset
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.indices = list(range(len(data)))  # set indices attribute
        print(self.data.keys())  
        print(f"Number of indices: {len(self.indices)}")


    def __getitem__(self, index):
        index = self.indices[index]  # get the actual index from self.indices
        text = self.data['text'][index]
        label = self.data['label'][index]
        embedded = self.data['embedded'][index]
        
        return text, label, embedded

    def __len__(self):
        return len(self.data)



In [4]:
# Define the pre-trained Visual-Bert model
model = VisualBertModel.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading: 100%|██████████| 445M/445M [02:42<00:00, 2.74MB/s] 
c:\Users\arman\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\arman\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at uclanlp/visual

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [6]:

# Define the pre-trained Visual-Bert model
model = VisualBertForPreTraining.from_pretrained('uclanlp/visualbert-vqa-coco-pre', num_labels=2)
#tokenizer = VisualBertTokenizer.from_pretrained('uclanlp/visualbert-vqa-coco-pre')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#possiblement a changer le VQA 

In [7]:
# Define the training loop

def train(model, tokenizer, train_dataset, optimizer, criterion, device, batch_size, epochs):
    model.to(device)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
    for epoch in range(epochs):
        running_loss = 0.0
        for text,label,embedded in train_loader:
            optimizer.zero_grad()
            text_encoded = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
            #image = embedded.to(device)
            text_encoded = {k: v.to(device) for k, v in text_encoded.items()}
            label = label.to(device)
            print(text_encoded['input_ids'].shape)
            print(embedded.shape)
            print(label.shape)
            #input_ids = text_encoded['input_ids'].unsqueeze(1)  # Add a new dimension at index 1
            #visual_embeds = embedded.unsqueeze(1).unsqueeze(2)
            #visual_embeds = embedded.unsqueeze(0).unsqueeze(0) # add two new dimensions at indices 0 and 1
            #visual_embeds = embedded.repeat(text_encoded['input_ids'].shape[0], 1, 1)
            #visual_embeds = embedded.view(-1, 1024) # reshape to (100, 1024) --> (2048, 768)

            outputs = model(input_ids=text_encoded['input_ids'], visual_embeds=embedded, labels=label)
            loss = criterion(outputs.logits, label)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        print('Epoch [%d] - loss: %.4f' % (epoch+1, running_loss/len(train_loader)))

#torch.from_numpy(embedded)

In [8]:

# Define the optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()


In [9]:
df=pd.read_json("data/train.jsonl",lines=True)

l = []

with open(r"C:\Users\arman\OneDrive\Bureau\data\file.pkl","rb") as f:
    l = pickle.load(f)
f.close()
df.loc[:, 'embedded'] = l

In [10]:
essai=df.head(100)

In [12]:
train_dataset = MyDataset(essai)
print(len(train_dataset))  # should print the length of your train data


Index(['id', 'img', 'label', 'text', 'embedded'], dtype='object')
Number of indices: 100
100


In [13]:
train(model, tokenizer, train_dataset, optimizer, criterion, device, batch_size=1, epochs=10)

torch.Size([1, 18])
torch.Size([1, 100, 1024])
torch.Size([1])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x1024 and 2048x768)